## Data Scraping - ScrapingAnt, Startup Nation

In [112]:
import pandas as pd

In [129]:
url = "https://raw.githubusercontent.com/LeoSoffiatti/nvoydia/main/ass/Startup_Nation_CSV.csv"
df = pd.read_csv(url)
df.head()


,Name,Finder URL,Description,Primary Sector,Founded,Employees,Funding Stage,Raised,Notes,Group,Group description
"=""Ruby Health""","=""https://finder.startupnationcentral.org/comp...","=""Value-based Care at Home""","=""Health Tech & Life Sciences""","=""2024""","=""1-10""","=""Seed""","=""Undisclosed""","=""""","=""""","=""""",NaN
"=""Seegnal eHealth""","=""https://finder.startupnationcentral.org/comp...","=""Patient-tailored Medicine Related Problems p...","=""Health Tech & Life Sciences""","=""2015""","=""11-50""","=""Seed""","=""Undisclosed""","=""""","=""""","=""""",NaN
"=""Senseera Health""","=""https://finder.startupnationcentral.org/comp...","=""A Single Blood Test for Broad Monitoring of ...","=""Health Tech & Life Sciences""","=""2020""","=""11-50""","=""Seed""","=""$7.1M""","=""""","=""""","=""""",NaN
"=""Sensifai Health""","=""https://finder.startupnationcentral.org/comp...","=""AI for Preemptive Digital Care""","=""Health Tech & Life Sciences""","=""2024""","=""1-10""","=""Seed""","=""Undisclosed""","=""""","=""""","=""""",NaN
"=""Shela Health ""","=""https://finder.startupnationcentral.org/comp...","=""Precision Women's Health Platform ""","=""Health Tech & Life Sciences""","=""2023""","=""1-10""","=""Seed""","=""$1M""","=""""","=""""","=""""",NaN


In [130]:
df = df.applymap(
    lambda x: x[2:-1] if isinstance(x, str) and x.startswith('="') and x.endswith('"') else x
)

C:\Users\emily\AppData\Local\Temp\ipykernel_95992\3680454134.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(


In [131]:
df.head()

,Name,Finder URL,Description,Primary Sector,Founded,Employees,Funding Stage,Raised,Notes,Group,Group description
"=""Ruby Health""",https://finder.startupnationcentral.org/compan...,Value-based Care at Home,Health Tech & Life Sciences,2024,1-10,Seed,Undisclosed,,,,NaN
"=""Seegnal eHealth""",https://finder.startupnationcentral.org/compan...,Patient-tailored Medicine Related Problems pre...,Health Tech & Life Sciences,2015,11-50,Seed,Undisclosed,,,,NaN
"=""Senseera Health""",https://finder.startupnationcentral.org/compan...,A Single Blood Test for Broad Monitoring of Hu...,Health Tech & Life Sciences,2020,11-50,Seed,$7.1M,,,,NaN
"=""Sensifai Health""",https://finder.startupnationcentral.org/compan...,AI for Preemptive Digital Care,Health Tech & Life Sciences,2024,1-10,Seed,Undisclosed,,,,NaN
"=""Shela Health """,https://finder.startupnationcentral.org/compan...,Precision Women's Health Platform,Health Tech & Life Sciences,2023,1-10,Seed,$1M,,,,NaN


In [136]:
import requests
from bs4 import BeautifulSoup

API_KEY = '71dbb3f4f77540d4967f97bd906841ef'
search_url = 'https://finder.startupnationcentral.org/startups?search=health'

params = {
    'url': search_url,
    'x-api-key': API_KEY,
    'render_js': 'true'  # Important for JS-rendered content
}

response = requests.get("https://api.scrapingant.com/v2/general", params=params)
html = response.text

# Now parse with BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')
startup_cards = soup.select('div[data-row-index]')  # general selector — may need refinement

for card in startup_cards:
    name = card.select_one('a span')  # startup name
    desc = card.select_one('div span:nth-of-type(2)')  # description
    print("Name:", name.text.strip() if name else "")
    print("Description:", desc.text.strip() if desc else "")
    print("---")


C:\Users\emily\AppData\Local\Temp\ipykernel_95992\1146264846.py:17: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(html, 'html.parser')


In [135]:
with open('out.html', 'w', encoding='utf-8') as f:
    f.write(html)


In [139]:
import requests
from bs4 import BeautifulSoup
import csv

# --- CHANGE THESE ---
API_KEY = '71dbb3f4f77540d4967f97bd906841ef'
DISCOVER_URL = 'https://www.crunchbase.com/discover/organization.companies/ec21c0c7a55b3f2b201bf51768f39036'
# --------------------

params = {
    'url': DISCOVER_URL,
    'x-api-key': API_KEY,
    'render_js': 'true',
    'wait_time': '10'  # seconds to wait for JS to load
}

res = requests.get('https://api.scrapingant.com/v2/general', params=params)
html = res.text

# Parse HTML
soup = BeautifulSoup(html, 'html.parser')

# ✅ DEBUG: Check what got loaded
with open('debug_crunchbase.html', 'w', encoding='utf-8') as f:
    f.write(html)

# Look for table rows — we may need to adjust if this fails
rows = soup.select('tbody tr')
print(f"Found {len(rows)} rows")

with open('crunchbase_healthcare_startups.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Name', 'Industry Groups', 'Location', 'Description', 'Stage'])

    for row in rows:
        cols = row.find_all('td')

        name = cols[0].get_text(strip=True) if len(cols) > 0 else ''
        industry = cols[1].get_text(strip=True) if len(cols) > 1 else ''
        location = cols[3].get_text(strip=True) if len(cols) > 3 else ''
        desc = cols[4].get_text(strip=True) if len(cols) > 4 else ''
        stage = cols[6].get_text(strip=True) if len(cols) > 6 else ''

        writer.writerow([name, industry, location, desc, stage])
        print(f"[✓] Scraped: {name}")


Found 0 rows


C:\Users\emily\AppData\Local\Temp\ipykernel_95992\2110112007.py:21: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(html, 'html.parser')
